'''
For open-set identification, we follow our previous work

It consists of two steps, enrollment and identification 

LFW, IJBC and VGG2

1.0 generate the deep features/ hash codes 
1.1 enrollement according to the protocol
1.2 identifcation 
2.0 performance evaluation

'''

In [8]:
from absl import app, flags, logging
from absl.flags import FLAGS
import os
import numpy as np
import tensorflow as tf
from modules.utils import set_memory_growth, load_yaml, l2_norm
from modules.models import ArcFaceModel, IoMFaceModelFromArFace, IoMFaceModelFromArFaceMLossHead,IoMFaceModelFromArFace2,IoMFaceModelFromArFace3,IoMFaceModelFromArFace_T,IoMFaceModelFromArFace_T1
import tqdm


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
# set_memory_growth()

cfg = load_yaml('./configs/iom_res50_random.yaml')  # cfg = load_yaml(FLAGS.cfg_path)
permKey = None
if cfg['head_type'] == 'IoMHead':  #
    # permKey = generatePermKey(cfg['embd_shape'])
    permKey = tf.eye(cfg['embd_shape'])  # for training, we don't permutate, won't influence the performance

arcmodel = ArcFaceModel(size=cfg['input_size'],
                        embd_shape=cfg['embd_shape'],
                        backbone_type=cfg['backbone_type'],
                        head_type='ArcHead',
                        training=False,
                        cfg=cfg)

ckpt_path = tf.train.latest_checkpoint('./checkpoints/arc_res50')
if ckpt_path is not None:
    print("[*] load ckpt from {}".format(ckpt_path))
    arcmodel.load_weights(ckpt_path)
else:
    print("[*] Cannot find ckpt from {}.".format(ckpt_path))
    exit()
m = cfg['m'] = 512
q = cfg['q'] = 4

# here I add the extra IoM layer and head
if cfg['hidden_layer_remark'] == '1':
    model = IoMFaceModelFromArFace(size=cfg['input_size'],
                                   arcmodel=arcmodel, training=False,
                                   permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == '2':
    model = IoMFaceModelFromArFace2(size=cfg['input_size'],
                                    arcmodel=arcmodel, training=False,
                                    permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == '3':
    model = IoMFaceModelFromArFace3(size=cfg['input_size'],
                                    arcmodel=arcmodel, training=False,
                                    permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == 'T':  # 2 layers
    model = IoMFaceModelFromArFace_T(size=cfg['input_size'],
                                     arcmodel=arcmodel, training=False,
                                     permKey=permKey, cfg=cfg)
elif cfg['hidden_layer_remark'] == 'T1':
    model = IoMFaceModelFromArFace_T1(size=cfg['input_size'],
                                      arcmodel=arcmodel, training=False,
                                      permKey=permKey, cfg=cfg)
else:
    model = IoMFaceModelFromArFace(size=cfg['input_size'],
                                   arcmodel=arcmodel, training=False,
                                   permKey=permKey, cfg=cfg)
model.summary(line_length=80)
cfg['embd_shape'] = m * q

[*] load ckpt from ./checkpoints/arc_res50\e_5_b_43044.ckpt
Model: "IoMface_model"
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_image (InputLayer)            [(None, 112, 112, 3)]           0           
________________________________________________________________________________
arcface_model (Model)               (None, 512)                     40375680    
________________________________________________________________________________
perm_layer (PermLayer)              (None, 512)                     0           
________________________________________________________________________________
IoMProjectionLayer (Model)          (None, 2048)                    1050624     
________________________________________________________________________________
IoMHead (Model)                     (None, 512)                     0           
Total params: 41,426,304
T

In [9]:
def load_data_from_dir(save_path, BATCH_SIZE=128, img_ext='png'):
    def transform_test_images(img):
        img = tf.image.resize(img, (112, 112))
        img = img / 255
        return img

    def get_label_withname(file_path):
        # convert the path to a list of path components
        parts = tf.strings.split(file_path, os.path.sep)
        # The second to last is the class-directory
        wh = parts[-2]
        return wh

    def process_path_withname(file_path):
        label = get_label_withname(file_path)
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = transform_test_images(img)
        return img, label

    list_gallery_ds = tf.data.Dataset.list_files(save_path + '/*/*.' + img_ext, shuffle=False)
    labeled_gallery_ds = list_gallery_ds.map(lambda x: process_path_withname(x))
    dataset = labeled_gallery_ds.batch(BATCH_SIZE)
    return dataset

def extractFeat(dataset, model, feature_dim):
    final_feature = np.zeros(feature_dim)
    feats = []
    names = []
    n = 0
    for image_batch, label_batch in tqdm.tqdm(dataset):
        feature = model(image_batch)
        for i in range(feature.shape[0]):
            n = n + 1
            feats.append(feature[i])
            mylabel = label_batch[i].numpy()
            names.append(mylabel)
            
    return feats,names,n


<DatasetV1Adapter shapes: (), types: tf.string>

In [6]:
dataset = load_data_from_dir('./data/lfw_mtcnnpy_160', BATCH_SIZE=128)


In [10]:
feats,names,n = extractFeat(dataset, model, m)

207it [20:36,  5.97s/it]


In [12]:
names

[b'AJ_Cook',
 b'AJ_Lamas',
 b'Aaron_Eckhart',
 b'Aaron_Guiel',
 b'Aaron_Patterson',
 b'Aaron_Peirsol',
 b'Aaron_Peirsol',
 b'Aaron_Peirsol',
 b'Aaron_Peirsol',
 b'Aaron_Pena',
 b'Aaron_Sorkin',
 b'Aaron_Sorkin',
 b'Aaron_Tippin',
 b'Abba_Eban',
 b'Abbas_Kiarostami',
 b'Abdel_Aziz_Al-Hakim',
 b'Abdel_Madi_Shabneh',
 b'Abdel_Nasser_Assidi',
 b'Abdel_Nasser_Assidi',
 b'Abdoulaye_Wade',
 b'Abdoulaye_Wade',
 b'Abdoulaye_Wade',
 b'Abdoulaye_Wade',
 b'Abdul_Majeed_Shobokshi',
 b'Abdul_Rahman',
 b'Abdulaziz_Kamilov',
 b'Abdullah',
 b'Abdullah',
 b'Abdullah',
 b'Abdullah',
 b'Abdullah_Ahmad_Badawi',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Gul',
 b'Abdullah_Nasseef',
 b'Abdullah_al-Attiyah',
 b'Abdullah_a